In [1]:
import requests, json, os
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
fpl_url = 'https://fantasy.premierleague.com/api/'

# get list of fields available at the bootstrap-static endpoint
requests.get(fpl_url+'bootstrap-static/').json().keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
# load player data
players = pd.json_normalize(
    requests.get(fpl_url + 'bootstrap-static/').json()['elements']
).drop(
    ['chance_of_playing_next_round',
     'chance_of_playing_this_round',
     'code',
     'cost_change_event',
     'cost_change_event_fall',
     'cost_change_start',
     'cost_change_start_fall',
     'ep_next',
     'ep_this',
     'event_points',
     'form',
     'news',
     'news_added',
     'photo',
     'selected_by_percent',
     'special',
     'squad_number',
     'status',
     'team_code',
     'transfers_in',
     'transfers_in_event',
     'transfers_out',
     'transfers_out_event',
     'value_form',
     'value_season',
     'corners_and_indirect_freekicks_order',
     'corners_and_indirect_freekicks_text',
     'direct_freekicks_order',
     'direct_freekicks_text',
     'penalties_order',
     'penalties_text'], axis=1
)

players.head()

,dreamteam_count,element_type,first_name,id,in_dreamteam,now_cost,points_per_game,second_name,team,total_points,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type
0,0,3,Mesut,1,False,68,0.0,Özil,1,0,Özil,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,604,240,604,240,604,240,604,240
1,0,2,Sokratis,2,False,49,0.0,Papastathopoulos,1,0,Sokratis,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,566,212,556,212,543,206,567,212
2,0,2,David,3,False,55,1.2,Luiz Moreira Marinho,1,7,David Luiz,364,0,0,0,7,0,0,0,0,0,0,0,52,72.0,23.1,22.0,11.7,239,93,270,79,267,78,297,100
3,0,3,Pierre-Emerick,4,False,115,3.4,Aubameyang,1,37,Aubameyang,986,2,1,3,14,0,0,0,2,0,0,1,123,141.4,170.8,277.0,58.9,135,51,42,30,31,15,36,20
4,0,2,Cédric,5,False,46,0.0,Soares,1,0,Cédric,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,542,206,529,206,504,197,543,206


In [4]:
# load team data
teams = pd.json_normalize(
    requests.get(fpl_url + 'bootstrap-static/').json()['teams']
).drop(
    ['code',
     'form',
     'pulse_id',
     'draw',
     'loss',
     'played',
     'points',
     'position',
     'team_division',
     'unavailable',
     'win'], axis=1
)

teams.head()

,id,name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,1,Arsenal,ARS,4,1190,1210,1170,1210,1190,1200
1,2,Aston Villa,AVL,3,1150,1160,1150,1150,1180,1210
2,3,Brighton,BHA,3,1080,1100,1150,1180,1090,1100
3,4,Burnley,BUR,2,1050,1080,1120,1190,1010,1030
4,5,Chelsea,CHE,4,1260,1280,1240,1280,1270,1310


In [4]:
# load position information
positions = pd.json_normalize(
    requests.get(fpl_url + 'bootstrap-static/').json()['element_types']
)[
    ['id',
     'plural_name',
     'plural_name_short',
     'singular_name'
    ]
]

positions.head()

,id,plural_name,plural_name_short,singular_name
0,1,Goalkeepers,GKP,Goalkeeper
1,2,Defenders,DEF,Defender
2,3,Midfielders,MID,Midfielder
3,4,Forwards,FWD,Forward


In [5]:
def get_gameweek_history(player_id):
    
    df = pd.json_normalize(
        requests.get(fpl_url + 'element-summary/' + str(player_id) + '/').json()['history']
    )
    
    return df

get_gameweek_history(1)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0
1,1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823
2,1,23,11,0,False,2020-09-28T19:00:00Z,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126
3,1,29,15,0,True,2020-10-04T13:00:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971
4,1,44,12,0,False,2020-10-17T16:30:00Z,1,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840
5,1,49,9,0,True,2020-10-25T19:15:00Z,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-3184,51023,276,3460
6,1,64,13,0,False,2020-11-01T16:30:00Z,0,1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-3147,48026,57,3204
7,1,69,2,0,True,2020-11-08T19:15:00Z,0,3,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-1691,46467,38,1729
8,1,82,10,0,False,2020-11-22T16:30:00Z,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-1364,45182,14,1378
9,1,89,20,0,True,2020-11-29T19:15:00Z,1,2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-525,44753,12,537


>**To do**: create dataframe of all player gameweek scores

In [6]:
# get gameweek history for one player
pd.merge(left=players[['id', 'web_name']],
         right=get_gameweek_history(1),
         left_on='id',
         right_on='element').head()

,id,web_name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,Özil,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0
1,1,Özil,1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823
2,1,Özil,1,23,11,0,False,2020-09-28T19:00:00Z,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126
3,1,Özil,1,29,15,0,True,2020-10-04T13:00:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971
4,1,Özil,1,44,12,0,False,2020-10-17T16:30:00Z,1,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840


In [7]:
def get_season_history(player_id):
    
    df = pd.json_normalize(
        requests.get(fpl_url + 'element-summary/' + str(player_id) + '/').json()['history_past']
    )
    
    df['id'] = player_id
    
    return df

pd.merge(left=players[['id', 'web_name']],
         right=get_season_history(1),
         on='id')

,id,web_name,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,1,Özil,2013/14,37605,100,96,137,2141,5,10,13,23,0,0,0,0,0,0,18,162,0.0,0.0,0.0,0.0
1,1,Özil,2014/15,37605,90,90,103,1857,4,6,9,16,0,0,0,0,0,0,13,511,0.0,0.0,0.0,0.0
2,1,Özil,2015/16,37605,85,92,200,3036,6,19,17,31,0,0,0,4,0,0,30,861,0.0,0.0,0.0,0.0
3,1,Özil,2016/17,37605,95,95,167,2841,8,11,12,30,0,0,0,2,0,0,19,729,834.4,1649.3,734.0,322.0
4,1,Özil,2017/18,37605,95,93,112,2161,4,9,8,36,0,0,0,4,0,0,11,506,640.6,1287.5,555.0,248.6
5,1,Özil,2018/19,37605,85,79,89,1732,5,3,5,24,0,0,0,2,0,0,9,363,429.6,714.2,254.0,140.0
6,1,Özil,2019/20,37605,75,69,53,1439,1,3,5,20,0,0,0,1,0,0,1,256,223.6,582.9,190.0,99.8
